In [103]:
# Instalación de librerías necesarias
!pip install asyncpraw
!pip install python-dotenv

In [47]:
from transformers import pipeline

# Modelo usado  ## TODO: Hacer pruebas con otros modelos o incluso dar la elección
emotion_model_path = "ayoubkirouane/BERT-Emotions-Classifier"

emotion_classifier = pipeline("text-classification", model=emotion_model_path, return_all_scores=True, padding=True, truncation=True)

pytorch_model.bin:  28%|##7       | 168M/606M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [106]:
import asyncpraw
from dotenv import load_dotenv
import os
# Leer el archivo de entorno con las credenciales
load_dotenv("credencialesPraw.env")

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user = os.getenv("REDDIT_USER_AGENT")

reddit = asyncpraw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user)


In [99]:
## Obtención de los comentarios del Post elegido
from praw.models import Comment
url = "https://www.reddit.com/r/nba/comments/1gxgtne/zion_williamson_shows_off_new_back_tattoo_and/"
submission = await reddit.submission(url=url)
comments = await submission.comments()
await comments.replace_more(limit=None)
comentarios = await comments.list()


In [100]:
# Función que dada una lista de comentarios, aplicamos el clasificador y nos quedamos la emoción más probable
def annotate_comments(comment_list,classifier=emotion_classifier):
    texts = [x.body for x in comment_list]
    results = classifier(texts)
    comments_dict = {}
    for comment, result in zip(comment_list, results):
        top = max(result, key=lambda x: x['score'])
        comments_dict[comment.id] = top['label']
        #comment['emotion_probs'] = result
    return comments_dict

# Recorremos los subreddits para aplicarle la función
comments_dict = annotate_comments(comentarios)


In [101]:
import polars as pl
# Uso de polars para agrupar por emocio y contar valores
polar = pl.DataFrame({'Comentario': list(comments_dict.keys()), 'Emoción': list(comments_dict.values())})
polar.group_by('Emoción').count()

<ipython-input-101-04bb897c1c3b>:4: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  polar.group_by('Emoción').count()


Emoción,count
str,u32
"""anger""",624
"""surprise""",15
"""optimism""",215
"""fear""",22
"""joy""",434
"""disgust""",201
"""anticipation""",438
"""sadness""",303
